# EDA
## Opis problemu
Nowotwór płuc jest wiodącą przyczyną śmierci nowotworowych na świecie. Odpowiednie wcześnie wykrycie symptonów (w naszym przypadku będą to guzki płuc) zwiększa szanse pacjenta na wyleczenie.

Wykrywanie guzków jest czasochłonne. Pojawia się więc pytanie czy możemy wykrywać guzki używając AI.

Prostym z pozoru rozwiązaniem jest utworzenie dużego datasetu. Nie jest to jednak proste zadanie gdyż jak zostało już stwierdzone wykrywanie guzków jest czasochłonne. Utworzenie dużego datasetu wymaga pracy wielu osób i dużych funduszy, żeby opłacić te osoby.

W naszym projekcie nie wnikniemy w wykrywanie guzków samych w sobie gdyż jest to problem zbyt zawiły i wykraczający znacznie poza zakres kursu "Uczenie Reprezentacji". Postaramy się natomiast sprawdzić czy model na postawie podanych koordynatów guzka jest w stanie powiedzieć czy to jest rzeczywiście guzek (co najmniej 3mm średnicy) czy nie jest to guzek (lub guzek jest poniżej 3mm średnicy). Można powiedzieć, że stworzymy weryfikator algorytmów do wykrywania guzków.

Porównamy różne metody self-supervised learning i zobaczymy czy pozwalają lepiej one rozróżnieć guzki od nie-guzków.

## Opis danych
Dane zostały wzięte ze strony https://www.kaggle.com/datasets/avc0706/luna16  
Pełny opis danych znajdziemy na tej stronie https://luna16.grand-challenge.org/

Nasze dane składają się z trzech części:
1. CT skanów, które przedstawiają płuca badanych osób
2. Plik zawierający wszystkie wykryte guzki (co najmniej 3 mm średnicy) potwierdzone przez co najmniej 3 radiologów
3. Plik zawierający wszystkie odrzucone guzki (brak wykrytego guzka lub guzek poniżej 3 mm średnicy) tzn. nie uzyskały weryfikacji co najmniej 3 radiologów

Każdy guzek i nie-guzek ma lokalizację w tzw. world coordinates. Jest to uniwersalnie przyjęta metoda zapisu lokalizacji rzeczy na skanach CT. Szczegóły czym są dokładnie te koordynaty znajdują się tutaj https://theaisummer.com/medical-image-coordinates/

WAŻNE - na danym skanie CT może być wiele guzków i nie-guzków

Warto zauważyć, że nasz zbiór danych został w całości zweryfikowany przez radiologów. Podczas porównywania metod self-supervised learning zasymulujemy jednak sytuację w której większość danych nie będzie polabelowana (tzn. będzie tylko predykcja lokalizacji potencjalnego guzka)

# Import bibliotek

In [ ]:
import numpy as np
import SimpleITK as sitk
import glob
import pandas as pd
import matplotlib.pyplot as plt

# Załadowanie pliku z annotacjami
Będziemy korzystać z subsetu 0. Jeżeli chcesz odpalić dalej notebook to musisz pobrać ten subset stąd https://drive.google.com/drive/folders/1KqiXAkoAolDVdupdfkGHstMMHg7JIRlT?usp=sharing i umieścić go w folderze projektu

In [ ]:
# Guzki zweryfikowane przez radiologów
df_ann = pd.read_csv('data/annotations.csv')
df_ann

In [ ]:
df_ann['seriesuid'].value_counts()

In [ ]:
# Guzki niezweryfikowane
df_irrelevant_findings = pd.read_csv('data/annotations_excluded.csv')
df_irrelevant_findings

In [ ]:
df_irrelevant_findings['seriesuid'].value_counts()

Każdy plik z annotacjami ma 5 kolumn:
1. seriesuid - numer identyfikacyjny pacjenta (pacjent może mieć więcej niż jeden skan CT
2. coordX - położenie guzka (oś X, world coordinates)
3. coordY - położenie guzka (oś Y, world coordinates)
4. coordZ - położenie guzka (oś Z, world coordinates)
5. diameter_mm - średnia guzka. W przypadku gdy nie ma guzka ta wartość wynosi -1. Na bazie tej wartości będziemy ustalać predykowaną klasę

# Złączenie danych i ustalenie danych pacjenta

In [ ]:
df = pd.concat([df_ann, df_irrelevant_findings])
df['class'] = df['diameter_mm'].apply(lambda x: 0 if x < 3 else 1)
df

In [ ]:
df['class'].value_counts()

Nasz dataset jest niezbalansowany. Widzimy znaczną przewagę nie-guzków nad guzkami. Będziemy musieli uważać, żeby nasz model nie nauczył się wykrywać samych nie-guzków

# Załadowanie obrazów
Załadujemy dwa obrazy, żeby zobaczyć jak wyglądają dane

In [ ]:
filepaths = glob.glob(f'data/images/subset0/*.mhd')
# filenames to także identyfikator klienta
filenames = [x.replace(f'data/images/subset0/', '').replace('.mhd', '') for x in filepaths]

In [ ]:
filepaths[:5]

In [ ]:
filenames[:5]

Do wczytana obrazów użyjemy biblioteki SimpleITK  
https://simpleitk.org/

In [ ]:
mhd_file1 = sitk.ReadImage(filepaths[0])
mhd_file1

In [ ]:
ct_scan1 = np.array(sitk.GetArrayFromImage(mhd_file1), dtype=np.float32)
ct_scan1.shape

In [ ]:
for i in range(0, 10):
    plt.imshow(ct_scan1[i], cmap='gray')
    plt.show()

In [ ]:
for i in range(0, 100, 10):
    plt.imshow(ct_scan1[i], cmap='gray')
    plt.show()

CT skany to tak naprawdę około 100 obrazów wymiaru 512x512, które pokazują nam płuca z różnych perspektyw

In [ ]:
mhd_file2 = sitk.ReadImage(filepaths[1])
ct_scan2 = np.array(sitk.GetArrayFromImage(mhd_file2), dtype=np.float32)
ct_scan2.shape

Mamy problem - każdy skan ma różną liczbę zdjęć. Możemy oczywiście próbować przetwarzać każdy obraz oddzielnie, ale znacząco wydłuży to trenowanie jak i samo użycie modelu będzie dłużej trwało. Dodatkowo kolejnym problemem będzie w jaki sposób odróżnić guzki od nie-guzków jeżeli na danym skanie może być kilka a nawet kilkanaście takich rzeczy wykrytych.

Warto jednak zauważyć jedną rzecz - większość z tych skanów nie jest nam potrzebna, bo nie pokazuje niczego wartego uwagi. Jeżeli bylibyśmy w stanie ograniczyć się tylko do najważniejszych skanów (i jak można wycinków z tych skanów) to jesteśmy w stanie rozwiązać problem nierównej liczby zdjęć i zmniejszyć wymiarowość danych.

Na pomoc przychodzą nam koordynaty w tzw. world coordinates. Bazując na nich jesteśmy w stanie zdobyć najbardziej istotne fragmenty ze skanów. Przetestujmy na naszych danych jak to będzie wyglądało

# Zmniejszenie wymiarowości danych

In [ ]:
# Wymiar do którego chcemy zmniejszyć nasze skany CT
DIMS_IRC = (10, 32, 32)

mhd_file = mhd_file1
ct_scan = ct_scan1
# Weź dane właściwego pacjenta. Bierzemy tylko jeden odczyt z danego pacjenta nawet jeżeli jest ich kilka
patient_data = df[df['seriesuid'] == filenames[0]].reset_index(drop=True).iloc[0]
patient_data

In [ ]:
# Bazowane na https://www.kaggle.com/code/mashruravi/pytorch-vs-cancer
def get_scan_chunk(ct_scan, patient_data):
    DIMS_IRC = (10, 32, 32)
    
    ct_scan.clip(-1000, 1000, ct_scan)
        
    origin_xyz = np.array(mhd_file.GetOrigin())
    voxel_size_xyz = np.array(mhd_file.GetSpacing())
    direction_matrix = np.array(mhd_file.GetDirection()).reshape(3, 3)
    
    coordX = patient_data['coordX']
    coordY = patient_data['coordY']
    coordZ = patient_data['coordZ']
    
    center_xyz = np.array([coordX, coordY, coordZ])
    
    cri = ((center_xyz - origin_xyz) @ np.linalg.inv(direction_matrix)) / voxel_size_xyz
    cri = np.round(cri)
    
    irc = (int(cri[2]), int(cri[1]), int(cri[0]))
    
    slice_list = []
    for axis, center_val in enumerate(irc):
        start_index = int(round(center_val - DIMS_IRC[axis]/2))
        end_index = int(start_index + DIMS_IRC[axis])
    
        if start_index < 0:
            start_index = 0
            end_index = int(DIMS_IRC[axis])
        
        if end_index > ct_scan.shape[axis]:
            end_index = ct_scan.shape[axis]
            start_index = int(ct_scan.shape[axis] - DIMS_IRC[axis])
            
        slice_list.append(slice(start_index, end_index))
    
    ct_scan_chunk = ct_scan[tuple(slice_list)]
    return ct_scan_chunk

ct_scan_chunk = get_scan_chunk(ct_scan, patient_data)
ct_scan_chunk.shape

Udało nam się zmniejszyć wymiarowość danych. Zobaczmy teraz jak obrazy wyglądają. Warto pamiętać, że akurat w tym przypadku wykryto guzek.

In [ ]:
for i in range(0, 10):
    plt.imshow(ct_scan_chunk[i], cmap='gray')
    plt.show()

Widzimy, że obrazy teraz są znacznie bardziej informatywne niż wcześniej. Model się skupi na nauce najważniejszych informacji, a nie całych skanów.

Zobaczmy ten sam proces przetwarzania danych, ale tym razem dla nie-guzka

In [ ]:
mhd_file = mhd_file2
ct_scan = ct_scan2
# Weź dane właściwego pacjenta. Bierzemy tylko jeden odczyt z danego pacjenta nawet jeżeli jest ich kilka
patient_data = df[df['seriesuid'] == filenames[1]].reset_index(drop=True).iloc[1]
patient_data

In [ ]:
ct_scan_chunk = get_scan_chunk(ct_scan, patient_data)
ct_scan_chunk.shape

Zobaczmy jak wyglądają skany gdy tego guzka nie ma

In [ ]:
for i in range(0, 10):
    plt.imshow(ct_scan_chunk[i], cmap='gray')
    plt.show()

Widzimy, że inaczej wyglądają oba skany. Znależliśmy więc efektywną metodę zmniejszenia wymiarowości danych tak żeby nie zatracić żadnej informacji

# Podsumowanie
Nasz problem jest tylko dotknięciem tematu wykrywania guzków płuc.

Jednak nasza analiza pozwoliła nam zauważyć kilka rzeczy:
1. Nasz problem jest niezbalansowany. Trzeba zwracać uwagę na to czy model rzeczywiście uczy się problemu czy tylko predykuje same nie-guzki
2. Warto uważać na potencjalny wyciek danych. Jeżeli zmieszamy dane o wszystkich guzkach i nie-guzkach to wówczas jest szansa, że ten sam pacjent znajdzie się w zbiorze treningowym i zbiorze walidacyjnym. Dla bezpieczeństwa najlepiej jeżeli jeden subset zostanie w całości przeznaczony na walidację (co też autorzy datasetu zalecają)
3. Mamy do czynienia z danymi medycznymi - trzeba bardzo uważać jak je przetwarzamy. Bardzo łatwo jest zrobić błąd w wyniku niewiedzy jak te dane działają
4. Jesteśmy w stanie znacząco zmniejszyć wymiarowość danych. Koniecznym krokiem będzie odpowiedni preprocessing tych danych, gdyż nie chcemy trzymać 60 GB danych na naszym dysku.